In [1]:
import numpy as np
import ase.io
import os
import subprocess
import pandas as pd

* Coordinates

In [2]:
file_coord = open("coord.raw", "w")
file_box = open("box.raw", "w")
file_type = open("type.raw", "w")

try:
    confs=ase.io.read("../traj.xyz",format='extxyz',index=":")
    print("Configuration readed!")
except:
    print("Configuration could not be read")
else:
    # Shuffle configurations randomly
    num_samples = len(confs)
    mask = np.random.permutation(num_samples)
    confs = [confs[i] for i in mask]
    # Write coord.raw and box.raw
    for conf in confs:
        file_coord.write(' '.join(conf.get_positions().flatten().astype('str').tolist()) + '\n')
        file_box.write(' '.join(conf.get_cell().flatten().astype('str').tolist()) + '\n')
    # Write type.raw only once using the first configuration
    atomic_numbers=confs[0].get_atomic_numbers()
    type_idx=[]
    for i in atomic_numbers:
        if i==8: type_idx.append(0)
        if i==1: type_idx.append(1)
    type_idx=np.array(type_idx)
    file_type.write(' '.join(type_idx.astype('str').tolist()))

file_coord.close()
file_box.close()
file_type.close()

Configuration readed!


* Energies

In [3]:
sim_file = pd.read_csv('../deepmd_jax.stdout', comment='#', sep='\\s+', skip_blank_lines=True)
sim_file = sim_file.drop_duplicates(subset=['Step'], ignore_index=True)  # Remove duplicate steps if any
energy = np.array(sim_file['PE'])
steps = np.array(sim_file['Step'])

energy = energy[steps%2000 == 0][1:]
energy = energy[mask]
np.savetxt("energy.raw", energy, fmt='%.6f')

print("Energy file created!")

Energy file created!


* Densities

In [4]:
cell=np.array(sim_file['box_x'])
number_O=64
number_H=64*2
totalmass=number_H*1.00784+number_O*15.9994
DENSITY_UNIT_CONVERSION=1.66054 #Conversion from uma/A³ to g/cm³

density=totalmass/cell[steps%2000 == 0][1:]**3 * DENSITY_UNIT_CONVERSION
density=density[mask]
np.savetxt("observable.raw", density, fmt='%.6f')

print("Observable file created!")

Observable file created!


Create sets

In [5]:
en = np.loadtxt('energy.raw', dtype=float)
obs = np.loadtxt('observable.raw', dtype=float)
assert len(confs) == len(en) == len(obs), "Files have different lengths!"

subprocess.run('./raw_to_set.sh')

nframe is 10000
nline per set is 2000
will make 5 sets
making set 0 ...
making set 1 ...
making set 2 ...
making set 3 ...
making set 4 ...


CompletedProcess(args='./raw_to_set.sh', returncode=0)